## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import pickle

from sklearn.metrics import classification_report

In [2]:
#!pip install mljar-supervised
from supervised.automl import AutoML

## Export data

In [4]:
with open('Saved data/X_train.pickle', 'rb') as data:
    X_train = pickle.load(data)
    
with open('Saved data/X_test.pickle', 'rb') as data:
    X_test = pickle.load(data)

with open('Saved data/y_train.pickle', 'rb') as data:
    y_train = pickle.load(data)
    
with open('Saved data/y_test.pickle', 'rb') as data:
    y_test = pickle.load(data)
    
with open('Saved data/labels.pickle', 'rb') as data:
    labels = pickle.load(data)

## AutoML compete mode 

In [4]:
automl = AutoML(mode="Compete")
automl.fit(X_train.to_numpy(), y_train.to_numpy().flatten())

Linear algorithm was disabled.
Disable stacking for split validation
AutoML directory: AutoML_1
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'LightGBM', 'Neural Network', 'Nearest Neighbors']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.59591 trained in 813.44 seconds
2_DecisionTree logloss 0.652404 trained in 860.55 seconds
3_DecisionTree logloss 0.652404 trained in 808.13 seconds
Skip default_algorithms because of the time limit.
Skip not_so_random because of the time limit.
Skip golden_features because no parameters were generated.
Skip insert_random_feature because no parameters were generated.
Skip features_selection because no parameters were genera

NameError: name 'X_test' is not defined

In [5]:
automlPredictions = automl.predict_all(X_test.to_numpy())
print(classification_report(y_test, automlPredictions['label'].to_numpy().flatten(), target_names=labels))

              precision    recall  f1-score   support

       Basal       0.95      0.90      0.92        20
        Her2       0.87      0.59      0.70        22
        LumA       0.83      0.96      0.89        68
        LumB       0.82      0.78      0.80        46

    accuracy                           0.85       156
   macro avg       0.87      0.81      0.83       156
weighted avg       0.85      0.85      0.84       156



## AutoML compete mode advanced

In [4]:
automlCompete = AutoML(
    mode="Compete",
    total_time_limit=3*3600,
    algorithms=["LightGBM", "Xgboost", "Extra Trees"],
    stack_models=True,
    train_ensemble=True,
    boost_on_errors=True,
    optuna_verbose=True,
    explain_level=2,
    ml_task = 'multiclass_classification',
)
automlCompete.fit(X_train.to_numpy(), y_train.to_numpy().flatten())

Disable stacking for split validation
AutoML directory: AutoML_3
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'Extra Trees']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_LightGBM logloss 0.462574 trained in 3580.5 seconds
* Step not_so_random will try to check up to 27 models
11_LightGBM logloss 0.35717 trained in 2746.73 seconds
There was an error during 2_Xgboost training.
Please check AutoML_3\errors.md for details.
Skip golden_features because no parameters were generated.
Skip insert_random_feature because no parameters were generated.
Skip features_selection because no parameters were generated.
*

AutoML(algorithms=['LightGBM', 'Xgboost', 'Extra Trees'], boost_on_errors=True,
       explain_level=2, ml_task='multiclass_classification', mode='Compete',
       stack_models=True, total_time_limit=10800)

In [5]:
automlCompetePredictions = automlCompete.predict_all(X_test.to_numpy())
print(classification_report(y_test, automlCompetePredictions['label'].to_numpy().flatten(), target_names=labels))

              precision    recall  f1-score   support

       Basal       1.00      0.95      0.97        20
        Her2       0.86      0.82      0.84        22
        LumA       0.93      1.00      0.96        68
        LumB       0.93      0.87      0.90        46

    accuracy                           0.93       156
   macro avg       0.93      0.91      0.92       156
weighted avg       0.93      0.93      0.93       156



In [6]:
automlCompete.report()

In [7]:
joblib.dump(automlCompete, 'Saved models & utils/automlCompete93strong.pkl')

['automlCompete93strong.pkl']